In [1]:
import numpy as np
import pandas as pd

from datasets import Dataset, DatasetDict
from transformers import AutoModelForSequenceClassification,AutoTokenizer, DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
import evaluate
import torch
from torch import softmax

## Load Model

In [2]:
model  = 'w11wo/indonesian-roberta-base-indonli'
tokz = AutoTokenizer.from_pretrained(model)

## Dataset IndoNLI

In [3]:
# Dataset IndoNLI
from datasets import load_dataset

dataset = load_dataset("indonli")

Found cached dataset indonli (C:/Users/Irsyadobby/.cache/huggingface/datasets/indonli/indonli/1.1.0/d34041bd1d1a555a4bcb4ffdb9fe904778da6f7c5343209fc1485dd68121cb62)


  0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 10330
    })
    validation: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 2197
    })
    test_lay: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 2201
    })
    test_expert: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 2984
    })
})

### Data Train

In [5]:
# Train Data
train = dataset['train']
train = pd.DataFrame(train)
train.head(10)

,premise,hypothesis,label
0,Presiden Joko Widodo (Jokowi) menyampaikan pre...,Prediksi akhir wabah tidak disampaikan Jokowi.,2
1,Meski biasanya hanya digunakan di fasilitas ke...,Masker sekali pakai banyak dipakai di tingkat ...,0
2,"Data dari Nielsen Music mencatat, ""Joanne"" tel...",Nielsen Music mencatat pada akhir minggu ini.,1
3,Album Wild West miliknya pada tahun 1981 merup...,Ia memiliki lebih dari satu album.,1
4,"Seperti namanya, paket internet sahur Telkomse...",Paket internet sahur tidak ditujukan untuk saa...,2
5,"Sebelumnya, Ada jatuh sakit pada 17 Maret, men...",Ada negatif mengidap COVID-19 secara tidak resmi.,2
6,"Sore ini, mayoritas indeks saham di Eropa berg...",Penurunan indeks DAX lebih besar dibandingkan ...,0
7,Perdana Menteri Inggris Theresa May diprediksi...,Perdana Menteri Inggris tidak kalah dalam Pemi...,1
8,"Pada tanggal itu, Horan bakal merilis album be...",Horan telah memiliki banyak album.,1
9,Banyak dari mereka yang juga mengucapkan selam...,Iko mengaku senang dengan pengalamannya tersebut.,1


In [6]:
train['input'] = 'TEXT 1: ' + train.premise + '; TEXT 2: ' + train.hypothesis 
train = train[['input','label']]

train_ds = Dataset.from_pandas(train)

# Tokenize input 
def tok_func(x) :
    return tokz(x['input'])

tok_train = train_ds.map(tok_func, batched = True)

Map:   0%|          | 0/10330 [00:00<?, ? examples/s]

### Data Test

In [9]:
test = dataset['validation']
test = pd.DataFrame(test)
test

,premise,hypothesis,label
0,Manuskrip tersebut berisi tiga catatan yang me...,Manuskrip tersebut tidak mencatat laporan kema...,2
1,"Dilansir dari Business Insider, menurut observ...",Tidak ada observasi yang pernah dilansir oleh ...,2
2,Perekonomian Jakarta terutama ditunjang oleh s...,Sektor jasa memberi pengaruh lebih besar darip...,1
3,Seorang wanita asal New York mengaku sangat be...,Tidak ada orang dari New York yang membenci ai...,2
4,Istana Bogor sangat cocok bagi Anda yang ingin...,Ada koleksi-koleksi bersejarah di Istana Bogor.,0
...,...,...,...
2192,"""Penguatan organ di tenggorokan dengan menggun...",Organ tenggorokan dapat dikuatkan menggunakan ...,0
2193,"Secara harfiah, kini orang tidak lagi perlu me...",Jas hujan payung merupakan cara bersiap-siap m...,1
2194,"""Berdasarkan data sebaran kasus positif corona...",100 pasien lainnya pada hari ini dinyatakan se...,1
2195,"""Gejala batuk pada COVID-19 biasanya adalah ba...",Gejala batuk pada COVID-19 hanya terjadi pada ...,2


In [10]:
test['input'] = 'TEXT 1: ' + test.premise + '; TEXT 2: ' + test.hypothesis
test = test[['input','label']]

test_ds = Dataset.from_pandas(test)

tok_test = test_ds.map(tok_func, batched = True)

Map:   0%|          | 0/2197 [00:00<?, ? examples/s]

## Training Model

In [11]:
# Training Parameter
bs = 16
epochs = 5
lr = 1e-5

In [12]:
# Evaluation Metric (Accuracy)
def compute_metrics(eval_pred):
    metric1 = evaluate.load("accuracy")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    accuracy = metric1.compute(predictions=predictions, references=labels)["accuracy"]

    return {"accuracy": accuracy}

In [13]:
# Training Arguments
args = TrainingArguments(
    'outputs',
    learning_rate = lr,
    warmup_ratio = 0.1,
    lr_scheduler_type = 'cosine',
    optim = "adamw_torch",
    fp16 = True,
    evaluation_strategy = "epoch",
    per_device_train_batch_size = bs,
    per_device_eval_batch_size = bs*2,
    num_train_epochs = epochs,
    weight_decay = 0.01,
    report_to = 'none'
)

In [15]:
# Training Model
model = AutoModelForSequenceClassification.from_pretrained(model, num_labels = 3)
trainer = Trainer(
    model,
    args,
    train_dataset = tok_train,
    eval_dataset = tok_test,
    tokenizer = tokz,
    compute_metrics = compute_metrics
)

In [16]:
trainer.train()

  0%|          | 0/3230 [00:00<?, ?it/s]

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


## Evaluation Dataset